In [1]:
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

import os

import glob
import pandas as pd
import numpy as np

import re
import requests
import json
import copy
import io

import xml.etree.ElementTree as ET


InteractiveShell.ast_node_interactivity = "all"

In [2]:
user = 'medien'
password = 'A4MV!ds2017rl'

url = 'https://service.bundeswahlleiter.de/medien/csv/kerg.csv'
download = requests.get(url, auth=(user, password))
decoded_content = download.content.decode('utf-8')


In [3]:
def getVotes(top_name, input_list):
    column_names = ['Nr', 'Gebiet', 'parent_node', str(top_name) + '_erststimmen_vorläufig', 
                         str(top_name) + '_erststimmen_vorperiode', str(top_name) + '_zweitstimmen_vorläufig', 
                         str(top_name) + '_zweitstimmen_vorperiode']
    standard_list = [0,1,2]
    select_cols = standard_list + input_list
    df = pd.read_csv(io.StringIO(decoded_content), sep=';', skiprows=5, usecols=select_cols, header=None, names=column_names)
    return df

## Wahlberechtigte (eligible voters)

In [63]:
meta_daten = df_wahlberechtigte[['Nr', 'Gebiet', 'parent_node']]

In [4]:
df_wahlberechtigte = getVotes('wahlberechtigte', [3,4,5,6])
df_wahlberechtigte = df_wahlberechtigte.dropna(how='all') 

In [5]:
df_wahlberechtigte.head()

,Nr,Gebiet,parent_node,wahlberechtigte_erststimmen_vorläufig,wahlberechtigte_erststimmen_vorperiode,wahlberechtigte_zweitstimmen_vorläufig,wahlberechtigte_zweitstimmen_vorperiode
0,1.0,Flensburg – Schleswig,1.0,225659.0,226944.0,225659.0,226944.0
1,2.0,Nordfriesland – Dithmarschen Nord,1.0,186384.0,186177.0,186384.0,186177.0
2,3.0,Steinburg – Dithmarschen Süd,1.0,175950.0,176731.0,175950.0,176731.0
3,4.0,Rendsburg-Eckernförde,1.0,199632.0,198903.0,199632.0,198903.0
4,5.0,Kiel,1.0,204650.0,205243.0,204650.0,205243.0


## Wähler (voters)

In [6]:
df_waehler = getVotes('waehler', [7,8,9,10])
df_waehler = df_waehler.dropna(how='all') 

## Ungültig (invalid votes)

In [7]:
df_ungueltig = getVotes('ungueltig', [11,12,13,14])
df_ungueltig = df_ungueltig.dropna(how='all') 

## Gueltig (valid voters)

In [8]:
df_gueltig = getVotes('gueltig', [15,16,17,18])
df_gueltig = df_gueltig.dropna(how='all') 

## Parties, CDU

In [9]:
df_cdu = getVotes('CDU', [19,20,21,22])
df_cdu = df_cdu.dropna(how='all') 

In [10]:
df_cdu.head()

,Nr,Gebiet,parent_node,CDU_erststimmen_vorläufig,CDU_erststimmen_vorperiode,CDU_zweitstimmen_vorläufig,CDU_zweitstimmen_vorperiode
0,1.0,Flensburg – Schleswig,1.0,68102.0,68235.0,58307.0,61347.0
1,2.0,Nordfriesland – Dithmarschen Nord,1.0,62260.0,64678.0,52933.0,56383.0
2,3.0,Steinburg – Dithmarschen Süd,1.0,54812.0,56669.0,47367.0,52408.0
3,4.0,Rendsburg-Eckernförde,1.0,66625.0,66775.0,56584.0,60349.0
4,5.0,Kiel,1.0,45691.0,47925.0,40011.0,43893.0


## Parties, SPD

In [11]:
df_spd = getVotes('SPD', [23,24,25,26])
df_spd = df_spd.dropna(how='all')

## Parties, Die Linke

In [12]:
df_linke = getVotes('Die Linke', [27,28,29,30])
df_linke = df_linke.dropna(how='all') 

## Parties, Grüne

In [13]:
df_gruene = getVotes('Grüne', [31,32,33,34])
df_gruene = df_gruene.dropna(how='all')

## Parties, CSU

In [14]:
df_csu = getVotes('CSU', [35,36,37,38])
df_csu = df_csu.dropna(how='all')

## Parties, FDP

In [15]:
df_fdp = getVotes('FDP', [39,40,41,42])
df_fdp = df_fdp.dropna(how='all')

## Parties, AfD

In [16]:
df_afd = getVotes('AfD', [43,44,45,46])
df_afd = df_afd.dropna(how='all')

## Dataframes for each Bundesland by party and also for Bundesgebiet (whole Germany)

In [17]:
def votesbyPartyBundesland(party_dataframe, column):
    df = party_dataframe.loc[(party_dataframe['Gebiet'] == 'Bundesgebiet') | (party_dataframe['parent_node'] == 99)]
    party = column.split('_')[0]
    df_short = df[['Gebiet', column]]
    df_short.columns = ['Bundesland', party]
    return df_short

In [18]:
afd = votesbyPartyBundesland(df_afd, 'AfD_zweitstimmen_vorperiode')
cdu = votesbyPartyBundesland(df_cdu, 'CDU_zweitstimmen_vorperiode')
spd = votesbyPartyBundesland(df_spd, 'SPD_zweitstimmen_vorperiode')
gruene = votesbyPartyBundesland(df_gruene, 'Grüne_zweitstimmen_vorperiode')
linke = votesbyPartyBundesland(df_linke, 'Die Linke_zweitstimmen_vorperiode')
fdp = votesbyPartyBundesland(df_fdp, 'FDP_zweitstimmen_vorperiode')
csu = votesbyPartyBundesland(df_csu, 'CSU_zweitstimmen_vorperiode')
gueltig = votesbyPartyBundesland(df_gueltig, 'gueltig_zweitstimmen_vorperiode')

In [19]:
votes_province_numbers = cdu.merge(csu,on='Bundesland').merge(spd,on='Bundesland').merge(gruene,on='Bundesland').merge(fdp,on='Bundesland').merge(linke,on='Bundesland').merge(afd,on='Bundesland').merge(gueltig, on='Bundesland')

votes_province_numbers = votes_province_numbers.fillna(0)

votes_province_numbers['CDU/CSU'] = votes_province_numbers['CDU'] + votes_province_numbers['CSU']

votes_province_numbers = votes_province_numbers.drop('CDU', 1)
votes_province_numbers = votes_province_numbers.drop('CSU', 1)

votes_province_numbers['Other'] = votes_province_numbers['gueltig'] - votes_province_numbers['CDU/CSU'] - votes_province_numbers['SPD'] - votes_province_numbers['FDP'] - votes_province_numbers['AfD'] - votes_province_numbers['Grüne'] - votes_province_numbers['Die Linke']

In [20]:
votes_province_numbers = votes_province_numbers.reindex([16,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

In [21]:
votes_province_percent = votes_province_numbers.copy()

votes_province_percent['CDU/CSU'] = votes_province_numbers['CDU/CSU'] * 100 / votes_province_numbers['gueltig']
votes_province_percent['SPD'] = votes_province_numbers['SPD'] * 100 / votes_province_numbers['gueltig']
votes_province_percent['FDP'] = votes_province_numbers['FDP'] * 100 / votes_province_numbers['gueltig']
votes_province_percent['Grüne'] = votes_province_numbers['Grüne'] * 100 / votes_province_numbers['gueltig']
votes_province_percent['Die Linke'] = votes_province_numbers['Die Linke'] * 100 / votes_province_numbers['gueltig']
votes_province_percent['AfD'] = votes_province_numbers['AfD'] * 100 / votes_province_numbers['gueltig']


In [22]:
def roundNumbers(x):
    try:
        return x.round(decimals=1)
    except:
        return x

In [23]:
votes_province_percent = votes_province_percent.apply(roundNumbers)
votes_province_percent['Other'] = 100 - votes_province_percent['CDU/CSU'] - votes_province_percent['SPD'] - votes_province_percent['FDP'] - votes_province_percent['AfD'] - votes_province_percent['Grüne'] - votes_province_percent['Die Linke']
votes_province_numbers = votes_province_numbers.drop('gueltig', 1)
votes_province_percent = votes_province_percent.drop('gueltig', 1)

In [24]:
#Change order of columns

votes_percent_export = votes_province_percent[['Bundesland', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD', 'Other']]
votes_numbers_export = votes_province_numbers[['Bundesland', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD', 'Other']]

In [25]:
votes_numbers_export.head(50)

,Bundesland,CDU/CSU,SPD,Grüne,Die Linke,FDP,AfD,Other
16,Bundesgebiet,18165446.0,11252215.0,3694057.0,3755699.0,2083533.0,2056985.0,2718921.0
0,Schleswig-Holstein,638756.0,513725.0,153137.0,84177.0,91714.0,74346.0,72435.0
1,Mecklenburg-Vorpommern,369048.0,154431.0,37716.0,186871.0,18968.0,48885.0,51824.0
2,Hamburg,285927.0,288902.0,112826.0,78296.0,42869.0,37142.0,44867.0
3,Niedersachsen,1825592.0,1470005.0,391901.0,223935.0,185647.0,165875.0,182305.0
4,Bremen,96459.0,117204.0,40014.0,33284.0,11204.0,12307.0,18940.0
5,Brandenburg,482601.0,321174.0,65182.0,311312.0,35365.0,83075.0,89653.0
6,Sachsen-Anhalt,485781.0,214731.0,46858.0,282319.0,30998.0,49661.0,68467.0
7,Berlin,508643.0,439387.0,220737.0,330507.0,63616.0,88060.0,136771.0
8,Nordrhein-Westfalen,3776563.0,3028282.0,760642.0,582925.0,498027.0,372258.0,479460.0


In [26]:
votes_percent_export.apply(roundNumbers).to_csv('votes_percent_2013.csv', index=False)
votes_numbers_export.to_csv('votes_numbers_2013.csv', index=False)

## Dataframe for each constituency by party (using first votes)

In [27]:
def votesbyPartyConstituency(party_dataframe, column):
    df = party_dataframe.loc[(party_dataframe['Gebiet'] != 'Bundesgebiet') & (party_dataframe['parent_node']  != 99) & (party_dataframe['Nr']  != 995) & (party_dataframe['Nr']  != 996)]
    party = column.split('_')[0]
    df_short = df[['Gebiet', column]]
    df_short.columns = ['Wahlkreis', party]
    return df_short

In [28]:
afd_const = votesbyPartyConstituency(df_afd, 'AfD_erststimmen_vorperiode')
cdu_const = votesbyPartyConstituency(df_cdu, 'CDU_erststimmen_vorperiode')
spd_const = votesbyPartyConstituency(df_spd, 'SPD_erststimmen_vorperiode')
gruene_const = votesbyPartyConstituency(df_gruene, 'Grüne_erststimmen_vorperiode')
linke_const = votesbyPartyConstituency(df_linke, 'Die Linke_erststimmen_vorperiode')
fdp_const = votesbyPartyConstituency(df_fdp, 'FDP_erststimmen_vorperiode')
csu_const = votesbyPartyConstituency(df_csu, 'CSU_erststimmen_vorperiode')
gueltig_const = votesbyPartyConstituency(df_gueltig, 'gueltig_erststimmen_vorperiode')

In [29]:
votes_constituency_numbers = cdu_const.merge(csu_const,on='Wahlkreis').merge(spd_const,on='Wahlkreis').merge(gruene_const,on='Wahlkreis').merge(fdp_const,on='Wahlkreis').merge(linke_const,on='Wahlkreis').merge(afd_const,on='Wahlkreis').merge(gueltig_const, on='Wahlkreis')

votes_constituency_numbers = votes_constituency_numbers.fillna(0)

votes_constituency_numbers['CDU/CSU'] = votes_constituency_numbers['CDU'] + votes_constituency_numbers['CSU']

votes_constituency_numbers = votes_constituency_numbers.drop('CDU', 1)
votes_constituency_numbers = votes_constituency_numbers.drop('CSU', 1)

votes_constituency_numbers['Other'] = votes_constituency_numbers['gueltig'] - votes_constituency_numbers['CDU/CSU'] - votes_constituency_numbers['SPD'] - votes_constituency_numbers['FDP'] - votes_constituency_numbers['AfD'] - votes_constituency_numbers['Grüne'] - votes_constituency_numbers['Die Linke']

In [30]:
votes_constituency_percent = votes_constituency_numbers.copy()

votes_constituency_percent['CDU/CSU'] = votes_constituency_numbers['CDU/CSU'] * 100 / votes_constituency_numbers['gueltig']
votes_constituency_percent['SPD'] = votes_constituency_numbers['SPD'] * 100 / votes_constituency_numbers['gueltig']
votes_constituency_percent['FDP'] = votes_constituency_numbers['FDP'] * 100 / votes_constituency_numbers['gueltig']
votes_constituency_percent['Grüne'] = votes_constituency_numbers['Grüne'] * 100 / votes_constituency_numbers['gueltig']
votes_constituency_percent['Die Linke'] = votes_constituency_numbers['Die Linke'] * 100 / votes_constituency_numbers['gueltig']
votes_constituency_percent['AfD'] = votes_constituency_numbers['AfD'] * 100 / votes_constituency_numbers['gueltig']

In [31]:
votes_constituency_percent = votes_constituency_percent.apply(roundNumbers)
votes_constituency_percent['Other'] = 100 - votes_constituency_percent['CDU/CSU'] - votes_constituency_percent['SPD'] - votes_constituency_percent['FDP'] - votes_constituency_percent['AfD'] - votes_constituency_percent['Grüne'] - votes_constituency_percent['Die Linke']
votes_constituency_numbers = votes_constituency_numbers.drop('gueltig', 1)
votes_constituency_percent = votes_constituency_percent.drop('gueltig', 1)

In [32]:
#Change order of columns

votes_constituency_percent_export = votes_constituency_percent[['Wahlkreis', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD', 'Other']]
votes_constituency_numbers_export = votes_constituency_numbers[['Wahlkreis', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD', 'Other']]

In [33]:
votes_constituency_percent_export.apply(roundNumbers).to_csv('votes_constituency_percent_2013.csv', index=False)
votes_constituency_numbers_export.to_csv('votes_constituency_numbers_2013.csv', index=False)

In [34]:
votes_constituency_percent_export.head(20)

,Wahlkreis,CDU/CSU,SPD,Grüne,Die Linke,FDP,AfD,Other
0,Flensburg – Schleswig,42.5,37.2,7.8,4.6,1.9,3.3,2.7
1,Nordfriesland – Dithmarschen Nord,49.8,32.1,6.5,3.6,2.4,3.1,2.5
2,Steinburg – Dithmarschen Süd,45.4,34.0,5.1,3.9,5.1,3.6,2.9
3,Rendsburg-Eckernförde,45.2,36.8,7.0,3.3,1.9,3.4,2.4
4,Kiel,33.1,43.0,10.0,5.3,2.1,2.8,3.7
5,Plön – Neumünster,43.7,37.5,6.4,3.8,1.9,3.9,2.8
6,Pinneberg,45.4,36.1,6.4,3.9,1.9,3.8,2.5
7,Segeberg – Stormarn-Mitte,45.4,35.3,6.1,3.9,2.1,3.8,3.4
8,Ostholstein – Stormarn-Nord,45.9,37.1,5.5,3.4,2.3,3.8,2.0
9,Herzogtum Lauenburg – Stormarn-Süd,45.2,34.6,7.0,3.8,2.2,4.6,2.6


### Processing for categorical map

In [35]:
constituency_without_Other = votes_constituency_percent_export[['Wahlkreis', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD']]

In [36]:
constituency_winner = constituency_without_Other.set_index('Wahlkreis').apply(lambda x: pd.Series(x.sort_values(ascending=False)
       .iloc[:3].index, index=[['winner', 'second', 'third']]), axis=1).reset_index()


In [37]:
constituency_winner_export = constituency_winner.merge(votes_constituency_percent_export, on="Wahlkreis")

In [38]:
import math

for index, row in constituency_winner_export.iterrows():
    winner_variable = row['winner']
    second_variable = row['second']
    third_variable = row['third']
    if(math.isnan(row[winner_variable])):
        constituency_winner_export.loc[index, 'winner'] = np.nan
    if(math.isnan(row[second_variable])):
        constituency_winner_export.loc[index, 'second'] = np.nan
    if(math.isnan(row[third_variable])):
        constituency_winner_export.loc[index, 'third'] = np.nan


In [39]:
constituency_winner_export

,Wahlkreis,winner,second,third,CDU/CSU,SPD,Grüne,Die Linke,FDP,AfD,Other
0,Flensburg – Schleswig,CDU/CSU,SPD,Grüne,42.5,37.2,7.8,4.6,1.9,3.3,2.7
1,Nordfriesland – Dithmarschen Nord,CDU/CSU,SPD,Grüne,49.8,32.1,6.5,3.6,2.4,3.1,2.5
2,Steinburg – Dithmarschen Süd,CDU/CSU,SPD,FDP,45.4,34.0,5.1,3.9,5.1,3.6,2.9
3,Rendsburg-Eckernförde,CDU/CSU,SPD,Grüne,45.2,36.8,7.0,3.3,1.9,3.4,2.4
4,Kiel,SPD,CDU/CSU,Grüne,33.1,43.0,10.0,5.3,2.1,2.8,3.7
5,Plön – Neumünster,CDU/CSU,SPD,Grüne,43.7,37.5,6.4,3.8,1.9,3.9,2.8
6,Pinneberg,CDU/CSU,SPD,Grüne,45.4,36.1,6.4,3.9,1.9,3.8,2.5
7,Segeberg – Stormarn-Mitte,CDU/CSU,SPD,Grüne,45.4,35.3,6.1,3.9,2.1,3.8,3.4
8,Ostholstein – Stormarn-Nord,CDU/CSU,SPD,Grüne,45.9,37.1,5.5,3.4,2.3,3.8,2.0
9,Herzogtum Lauenburg – Stormarn-Süd,CDU/CSU,SPD,Grüne,45.2,34.6,7.0,3.8,2.2,4.6,2.6


In [40]:
constituency_winner_export.apply(roundNumbers).to_csv('constituency_winner.csv', index=False)

# XML Import for seats

In [41]:
url_xml = 'https://service.bundeswahlleiter.de/medien/daten/sitze_02.xml'
download = requests.get(url_xml, auth=(user, password))
decoded_content = download.content.decode('utf-8')
xml_data = re.sub('\s+',' ', decoded_content)
root = ET.fromstring(xml_data)

In [42]:
def parseXML(xml_data):
    gebietsnummer = []
    partei = []
    sitze = []
    for gebiet in xml_data.findall('Gebiet'):
        for gruppe in gebiet.findall('SitzeGruppe'):
            gebietsnummer.append(gebiet.attrib['Gebietsnummer'])
            partei.append(gruppe.get('Gruppe'))
            sitze.append(gruppe.get('Gesamt'))
            
    return pd.DataFrame({'gebietsnummer': gebietsnummer,
                         'partei': partei,
                         'sitze': sitze})

In [43]:
seats = parseXML(root)

In [88]:
def matchProvince(s):
    if s == '99':
        return 'bundesweit'
    elif s == '1.0':
        return 'Schleswig-Holstein'
    elif s == '13.0':
        return 'Mecklenburg-Vorpommern'
    elif s == '2.0':
        return 'Hamburg'
    elif s == '3.0':
        return 'Niedersachsen'
    elif s == '4.0':
        return 'Bremen'
    elif s == '12.0':
        return 'Brandenburg'
    elif s == '15.0':
        return 'Sachsen-Anhalt'
    elif s == '11.0':
        return 'Berlin'
    elif s == '5.0':
        return 'Nordrhein-Westfalen'
    elif s == '14.0':
        return 'Sachsen'
    elif s == '6.0':
        return 'Hessen'
    elif s == '16.0':
        return 'Thüringen'
    elif s == '7.0':
        return 'Rheinland-Pfalz'
    elif s == '9.0':
        return 'Bayern'
    elif s == '8.0':
        return 'Baden-Württemberg'
    elif s == '10.0':
        return 'Saarland'
    return ''

In [49]:
def matchParty(s):
    if s == '2':
        return 'CDU'
    elif s == '1':
        return 'SPD'
    elif s == '34':
        return 'Die Linke'
    elif s == '5':
        return 'Grüne'
    elif s == '3':
        return 'CSU'
    elif s == '4':
        return 'FDP'
    elif s == '35':
        return 'AfD'
    return ''

In [50]:
seats['gebiet'] = seats['gebietsnummer'].apply(matchProvince)
seats['partei_match'] = seats['partei'].apply(matchParty)

In [51]:
seats_pivot = seats.pivot(index='gebiet', columns='partei_match', values='sitze')
seats_pivot_reset = seats_pivot.reset_index()

In [52]:
seats_pivot_reset['CDU'] = pd.to_numeric(seats_pivot_reset['CDU'], errors='ignore')
seats_pivot_reset['SPD'] = pd.to_numeric(seats_pivot_reset['SPD'], errors='ignore')
seats_pivot_reset['CSU'] = pd.to_numeric(seats_pivot_reset['CSU'], errors='ignore')
seats_pivot_reset['Die Linke'] = pd.to_numeric(seats_pivot_reset['Die Linke'], errors='ignore')
seats_pivot_reset['Grüne'] = pd.to_numeric(seats_pivot_reset['Grüne'], errors='ignore')

seats_pivot_reset = seats_pivot_reset.fillna(0)

In [53]:
seats_pivot_reset['CDU/CSU'] = seats_pivot_reset['CDU'] + seats_pivot_reset['CSU']

In [92]:
seats_export = seats_pivot_reset[['gebiet', 'CDU/CSU', 'SPD', 'Grüne', 'Die Linke', 'FDP', 'AfD']]

In [93]:
def convert(x):
    try:
        return x.astype(int)
    except:
        return x

In [94]:
seats_export.apply(convert).to_csv('seats_2017.csv', index=False)

# XML Import for elected PMs

In [58]:
url_xml = 'https://service.bundeswahlleiter.de/medien/daten/gewaehlte_02.xml'
download = requests.get(url_xml, auth=(user, password))
decoded_content = download.content.decode('utf-8')
xml_data = re.sub('\s+',' ', decoded_content)
root = ET.fromstring(xml_data)

In [59]:
def parsePM(xml_data):
    vorname = []
    name = []
    gebietsnummer = []
    gruppe = []
    gebietsart = []
    geburtsdatum = []
    beruf = []
    vorperiode = []
    wahlart = []
    geschlecht = []
    titel = []
    namenszusatz = []
    for kandidat in xml_data.findall('Kandidat'):
        for person in kandidat.findall('Personendaten'):
            name.append(person.get('Name'))
            vorname.append(person.get('Vorname'))
            geburtsdatum.append(person.get('Geburtsdatum'))
            beruf.append(person.get('Beruf1'))
            geschlecht.append(person.get('Geschlecht'))
            titel.append(person.get('Titel'))
            namenszusatz.append(person.get('Namensbestandteile'))
        for wahldaten in kandidat.findall('Wahldaten'):
            wahlart.append(wahldaten.get('Gewaehlt'))
            vorperiode.append(wahldaten.get('GewaehltVorp'))
        for wahldaten in kandidat.findall('Wahldaten/Direkt'):
            gebietsnummer.append(wahldaten.get('Gebietsnummer'))
            gruppe.append(wahldaten.get('Gruppe'))
            gebietsart.append(wahldaten.get('Gebietsart'))
        for wahldaten in kandidat.findall('Wahldaten/Liste'):
            gebietsnummer.append(wahldaten.get('Gebietsnummer'))
            gruppe.append(wahldaten.get('Gruppe'))
            gebietsart.append(wahldaten.get('Gebietsart'))
            
    return pd.DataFrame({'gebietsnummer': gebietsnummer,
                         'vorname': vorname,
                         'name': name,
                         'geschlecht': geschlecht,
                         'gruppe': gruppe,
                         'gebietsart': gebietsart,
                         'beruf': beruf,
                         'geburtsdatum': geburtsdatum,
                         'wahlart': wahlart,
                         'vorperiode': vorperiode,
                         'titel': titel,
                         'namenszusatz': namenszusatz})
        

In [60]:
df_pm = parsePM(root)

In [61]:
df_pm.head()

,beruf,gebietsart,gebietsnummer,geburtsdatum,geschlecht,gruppe,name,namenszusatz,titel,vorname,vorperiode,wahlart
0,Dipl.-Agraringenieur / Angestellter ö. D. / Un...,WAHLKREIS,7,1952,m,2,Abercron,von,Dr.,Michael,N,E
1,Angestellte,LAND,4,1976,w,34,Achelwilm,None,None,Doris Maria,N,Z
2,Vermögensverwalter,LAND,3,1965,m,4,Aggelidis,None,None,Grigorios,N,Z
3,Sozialwissenschaftlerin,LAND,8,1982,w,34,Akbulut,None,None,Gökay,N,Z
4,MdB / Geschäftsführer,LAND,3,1968,m,2,Albani,None,None,Stephan,J,Z


In [64]:
wahlkreis_match = df_pm.loc[df_pm['gebietsart'] == 'WAHLKREIS']
land_match = df_pm.loc[df_pm['gebietsart'] == 'LAND']
meta_daten_wahlkreis = meta_daten.loc[meta_daten['parent_node'] != 99]
meta_daten_land = meta_daten.loc[meta_daten['parent_node'] == 99]

In [65]:
wahlkreis_match['gebietsnummer'] = pd.to_numeric(wahlkreis_match['gebietsnummer'], errors='ignore')
land_match['gebietsnummer'] = pd.to_numeric(land_match['gebietsnummer'], errors='ignore')

/Users/felix/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/felix/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [66]:
wahlkreis_match_merged = wahlkreis_match.merge(meta_daten_wahlkreis, left_on="gebietsnummer", right_on="Nr")

In [67]:
land_match_merged = land_match.merge(meta_daten_land, left_on="gebietsnummer", right_on="Nr")

In [68]:
land_match_merged = land_match_merged[['name', 'vorname','Nr']]

In [69]:
wahlkreis_match_merged = wahlkreis_match_merged[['name', 'vorname','parent_node', 'Gebiet']]

In [70]:
df_pms_merged = df_pm.merge(wahlkreis_match_merged, left_on=['name', 'vorname'], right_on=['name', 'vorname'], how="left")

In [71]:
df_pms_merged_2 = df_pms_merged.merge(land_match_merged, left_on=['name', 'vorname'], right_on=['name', 'vorname'], how="left")

In [72]:
df_pms_merged_2 = df_pms_merged_2.fillna(0)

In [73]:
df_pms_merged_2['bundesland_kennung'] = df_pms_merged_2['Nr'] + df_pms_merged_2['parent_node']

In [74]:
df_pms_merged_2 = df_pms_merged_2.drop('Nr', 1)
df_pms_merged_2 = df_pms_merged_2.drop('parent_node', 1)

In [85]:
df_pms_merged_2['bundesland_kennung'] = df_pms_merged_2['bundesland_kennung'].astype('str')

In [89]:
df_pms_merged_2['bundesland'] = df_pms_merged_2['bundesland_kennung'].apply(matchProvince)

In [77]:
def matchPartyCDU_CSU(s):
    if s == '2':
        return 'CDU/CSU'
    elif s == '1':
        return 'SPD'
    elif s == '34':
        return 'Die Linke'
    elif s == '5':
        return 'Grüne'
    elif s == '3':
        return 'CDU/CSU'
    elif s == '4':
        return 'FDP'
    elif s == '35':
        return 'AfD'
    return ''

In [78]:
df_pms_merged_2['party'] = df_pms_merged_2['gruppe'].apply(matchPartyCDU_CSU)

In [91]:
df_pms_merged_2.to_csv('parliament_members_2017.csv', index=False)

In [90]:
df_pms_merged_2.head()

,beruf,gebietsart,gebietsnummer,geburtsdatum,geschlecht,gruppe,name,namenszusatz,titel,vorname,vorperiode,wahlart,Gebiet,bundesland_kennung,bundesland,party
0,Dipl.-Agraringenieur / Angestellter ö. D. / Un...,WAHLKREIS,7,1952,m,2,Abercron,von,Dr.,Michael,N,E,Pinneberg,1.0,Schleswig-Holstein,CDU/CSU
1,Angestellte,LAND,4,1976,w,34,Achelwilm,0,0,Doris Maria,N,Z,0,4.0,Bremen,Die Linke
2,Vermögensverwalter,LAND,3,1965,m,4,Aggelidis,0,0,Grigorios,N,Z,0,3.0,Niedersachsen,FDP
3,Sozialwissenschaftlerin,LAND,8,1982,w,34,Akbulut,0,0,Gökay,N,Z,0,8.0,Baden-Württemberg,Die Linke
4,MdB / Geschäftsführer,LAND,3,1968,m,2,Albani,0,0,Stephan,J,Z,0,3.0,Niedersachsen,CDU/CSU


# Get pictures of PMs

In [825]:
pm_name_data = df_pm[['name', 'vorname', 'titel', 'namenszusatz']]

In [822]:
pm_name_data['name'] = pm_name_data['name'].str.lower()
pm_name_data['vorname'] = pm_name_data['vorname'].str.lower()
pm_name_data['titel'] = pm_name_data['titel'].str.lower()
pm_name_data['namenszusatz'] = pm_name_data['namenszusatz'].str.lower()
pm_name_data = pm_name_data.apply(lambda x: x.str.replace('.',''))

In [823]:
pm_name_data['seach_key'] = pm_name_data['titel'] + "-" + pm_name_data['vorname'] + "-" + pm_name_data['namenszusatz'] + "-" + pm_name_data['name']



In [824]:
pm_name_data

,name,vorname,titel,namenszusatz,seach_key
0,abercron,michael,dr,von,dr-michael-von-abercron
1,achelwilm,doris maria,,,-doris maria- -achelwilm
2,akbulut,gökay,,,-gökay- -akbulut
3,albani,stephan,,,-stephan- -albani
4,albsteiger,katrin,,,-katrin- -albsteiger
5,altenkamp,norbert maria,,,-norbert maria- -altenkamp
6,altmaier,peter,,,-peter- -altmaier
7,amthor,philipp,,,-philipp- -amthor
8,amtsberg,luise,,,-luise- -amtsberg
9,andreae,kerstin,,,-kerstin- -andreae


# Facebook analytics

In [119]:
import facebook
import json

In [234]:
#afd
page_ID = 'alternativefuerde'

base='https://graph.facebook.com/v2.10'
node='/'+ page_ID + '/insights/page_fans_country?since=2017-09-01'
url=base+node
parameters={'period':'lifetime','access_token':'EAACEdEose0cBANROVtrzeaA1H1Q8ZB8Jfy5EpYUQpFUpreac6VEsKdJ4pXBAguzlXEOZAFUmR5IS90ZCA44kR2GW5DPPtyD7NwbTb2rgW71qqFtOzZBpnUZBw2k38HOyEgBkCxFJNI5xXx3gEiY1OmylmCkZBlqTj6w1O1shE3zHpSZCHGD3tkWigzi1quhuvIZD'}
object=requests.get(url,params=parameters).text.encode('utf-8')
data=json.loads(object)
fb_data = data['data'][0]['values']

In [236]:
fb_df

= data['data'][0]['values'][0]

17

In [166]:
sum(value for key, value in fb_data.items())

355641

# Candidates

In [211]:
btw17_kandidaten = pd.read_csv('bundeswahlleiter/btw17-kandidaten.csv')

In [212]:
def matchPartyCandidates(s):
    if s == 'CDU':
        return 'CDU/CSU'
    elif s == 'SPD':
        return 'SPD'
    elif s == 'Die Linke':
        return 'Die Linke'
    elif s == 'Grüne':
        return 'Grüne'
    elif s == 'CSU':
        return 'CDU/CSU'
    elif s == 'FDP':
        return 'FDP'
    elif s == 'AfD':
        return 'AfD'
    else:
        return 'Other'

In [213]:
btw17_kandidaten['partei'] = btw17_kandidaten['partei'].apply(matchPartyCandidates)

In [214]:
btw17_kandidaten = btw17_kandidaten.drop('geburtsort', 1)
btw17_kandidaten = btw17_kandidaten.drop('berufsgruppe', 1)
btw17_kandidaten = btw17_kandidaten.drop('wahlkreis_id', 1)
btw17_kandidaten = btw17_kandidaten.drop('liste_bundesland_land', 1)
btw17_kandidaten = btw17_kandidaten.drop('liste_partei', 1)
btw17_kandidaten = btw17_kandidaten.drop('liste_partei_lang', 1)
btw17_kandidaten = btw17_kandidaten.drop('mdb', 1)
btw17_kandidaten = btw17_kandidaten.drop('lat', 1)
btw17_kandidaten = btw17_kandidaten.drop('lng', 1)
btw17_kandidaten = btw17_kandidaten.drop('id_eigen', 1)
btw17_kandidaten = btw17_kandidaten.drop('foto', 1)
btw17_kandidaten = btw17_kandidaten.drop('scatter_x', 1)
btw17_kandidaten = btw17_kandidaten.drop('scatter_y', 1)
btw17_kandidaten = btw17_kandidaten.drop('color', 1)
btw17_kandidaten = btw17_kandidaten.drop('liste_platz', 1)
btw17_kandidaten = btw17_kandidaten.drop('id', 1)
btw17_kandidaten = btw17_kandidaten.drop('nachname_ohne', 1)
btw17_kandidaten = btw17_kandidaten.drop('wohnort', 1)
btw17_kandidaten = btw17_kandidaten.drop('namenszusatz', 1)

In [215]:
def matchBundesland(s):
    if s == 'BW':
        return 'Baden-Württemberg'
    elif s == 'BY':
        return 'Bayern'
    elif s == 'BE':
        return 'Berlin'
    elif s == 'BB':
        return 'Brandenburg'
    elif s == 'HB':
        return 'Bremen'
    elif s == 'HH':
        return 'Hamburg'
    elif s == 'HE':
        return 'Hessen'
    elif s == 'MV':
        return 'Mecklenburg-Vorpommern'
    elif s == 'NI':
        return 'Niedersaschsen'
    elif s == 'NW':
        return 'Nordrhein-Westfalen'
    elif s == 'RP':
        return 'Rheinland-Pfalz'
    elif s == 'SL':
        return 'Saarland'
    elif s == 'SN':
        return 'Sachsen'
    elif s == 'ST':
        return 'Sachsen-Anhalt'
    elif s == 'SH':
        return 'Schleswig-Holstein'
    elif s == 'TH':
        return 'Thüringen'

In [216]:
btw17_kandidaten['bundesland'] = btw17_kandidaten['bundesland'].apply(matchBundesland)

In [217]:
btw17_kandidaten.to_csv('candidates_2017.csv', index=False)